In [3]:
import numpy as np
import pandas as pd 
from scipy import spatial
from glob import glob

In [2]:
positive_df = pd.read_csv('pos_data(no traffic attribute).csv', encoding='utf-8-sig', low_memory=False)

In [3]:
positive_df = positive_df.head(10)

In [4]:
negitive_df = pd.read_csv('neg_data.csv', encoding='utf-8-sig', low_memory=False)

In [5]:
negitive_df

,y,x,Type,name,location,tt,年,月,日,時
0,23.703466,120.536702,Road,北港 (Beigang),雲林縣斗六市鎮北路345-1號(斗六消防分隊),1608276768,2020,12,18,15
1,25.077870,121.513505,freeway,天母 (Tianmu),臺北市士林區天母東路116號(三玉國小內),1620221758,2021,5,5,21
2,24.949437,121.663959,provincialRoad,火燒寮 (Huoshaoliao),新北市平溪區東勢里,1634912335,2021,10,22,22
3,24.633933,121.279105,countryRoad,信義 (Xinyi),臺北市信義區市府路1號(台北市政府頂樓),1640055898,2021,12,21,11
4,22.680644,120.489808,Road,赤山 (Chishan),屏東縣屏東市復興路245號(台灣糖業公司屏東區處場區內),1554611431,2019,4,7,12
...,...,...,...,...,...,...,...,...,...,...
3999995,22.551515,120.620479,countyRoad,春日 (Chunri),屏東縣萬巒鄉新隆路64號萬巒農場（佳佐段180號）,1571036363,2019,10,14,14
3999996,23.925731,120.711020,countryRoad,梅峰 (Meifeng),南投縣南投市龍泉里玉井街5號(南投市公所樓頂),1655889867,2022,6,22,17
3999997,23.759012,120.576133,Road,莿桐 (Citong),雲林縣林內鄉中正路685號(林內消防分隊),1600828133,2020,9,23,10
3999998,24.032694,120.691294,Road,昆陽 (Kunyang),南投縣草屯鎮中山里草鞋墩1街8號(草屯鎮公所樓頂),1662136740,2022,9,3,0


In [6]:
negitive_df = negitive_df[['年','月','日','時','x','y','name','location']]

In [7]:
negitive_df = negitive_df.rename(columns=dict(zip(['年','月','日','時','x','y','name','location'], list(positive_df.columns[0:6])+['name','location'])))

# 合併各年分的天氣站資料

In [8]:
### 合併各年份氣象站天氣資料
years = [2018, 2019, 2020, 2021,2022]
weather = None
for year in years:        
    if str(type(weather))!="<class 'pandas.core.frame.DataFrame'>":
        weather = pd.read_csv(f'./合併天氣/weather{year}.csv',encoding='utf-8-sig', low_memory=False)
        print(f'已讀取{year}年天氣資料')
        continue 
    new = pd.read_csv(f'./合併天氣/weather{year}.csv',encoding='utf-8-sig', low_memory=False)
    weather = pd.concat([weather, new], axis=0)
    print(f'已讀取{year}年天氣資料')

# 合併三個最近天氣站的天氣資料
print('開始合併各天氣站的天氣資料')
weather['date'] = weather['date'].str.findall('\d+').str.join('')
weather['觀測時間(hour)'] = weather['觀測時間(hour)'].astype(str).apply(lambda x: '0' + x if len(x) <2 else '00' if x == '24' else x)
weather['weather_concat'] = weather['date'] + weather['觀測時間(hour)'] + weather['location']
weather = weather.drop_duplicates(subset=['weather_concat'])

已讀取2018年天氣資料
已讀取2019年天氣資料
已讀取2020年天氣資料
已讀取2021年天氣資料
已讀取2022年天氣資料
開始合併各天氣站的天氣資料


# 計算三個最近的天氣站
    -把最近的三個站點的距離、名稱，合併到事故資料

In [9]:
def weatherCombine(data:pd.DataFrame,weather:pd.DataFrame)->pd.DataFrame:
#     print('開始合併天氣資料')
    
#     ### 合併各年份氣象站天氣資料
#     years = [2018, 2019, 2020, 2021,2022]
#     weather = None
#     for year in years:        
#         if str(type(data))!="<class 'pandas.core.frame.DataFrame'>":
#             weather = pd.read_csv(f'./合併天氣/weather{year}.csv',encoding='utf-8-sig', low_memory=False)
#             print(f'已讀取{year}年天氣資料')
#             continue 
#         new = pd.read_csv(f'./合併天氣/weather{year}.csv',encoding='utf-8-sig', low_memory=False)
#         weather = pd.concat([weather, new], axis=0)
#         print(f'已讀取{year}年天氣資料')
        
    print('開始讀取天氣站名與站點位置')
    ### 取出天氣站的站名與站點
    station_has_weather = pd.DataFrame({'location':weather['location'].unique()}).merge( weather.groupby('location')[['longitude','latitude']].mean(), how='left',on='location')
    origin_station_data = pd.read_csv('./合併天氣/stationData.csv', low_memory=False)[['name','location']]
    station_has_weather = station_has_weather.merge(origin_station_data, how='left', left_on='location', right_on='name')[['location_x', 'longitude', 'latitude','location_y']]
    station_has_weather.columns = ['name','long','lat','location']
    
    print('已取出有天氣資料的站點，以及站點的位置資料')
    
    #station
    station = station_has_weather
    station_has_weather = None

    
    print('開始將有天氣資料的站點建立KDtree')
    ### 計算事故點最近的三個氣象站
    KDTree = spatial.KDTree(station[['long','lat']])
    print('開始計算每個車禍點最近的3個氣象站點')
    global c
    c=0
    def nearest_weather_station(x: pd.DataFrame):
        TreeResult = KDTree.query((x.經度, x.緯度),k=3)
        distance = TreeResult[0]  #[測站1距離(經度*緯度),測站2距離(經度*緯度),測站3距離(經度*緯度)]
        index = TreeResult[1]     #[測站1在station的index,測站2在station的index, 測站3在station的index]

        first_station = station.iloc[index[0], [0,3]]     #[0,3]的column為測站名稱、測站地址
        second_station = station.iloc[index[1], [0,3]]
        third_station = station.iloc[index[2], [0,3]]

        first_distance = distance[0]
        second_distance = distance[1]
        third_distance = distance[2]

        ser = pd.Series([first_station[0],first_distance,second_station[0],second_distance,third_station[0],third_distance])

        global c
        c+=1
        if c%100000 == 0:
            print(f"車禍最近的3個氣象站點:已計算{c}筆| ",end='' )

        return ser

    data = pd.concat([data, data.apply(nearest_weather_station, axis=1)], axis = 1)
    data = data.rename(columns={0:'氣象站1',
                                1:'氣象站1距離',
                                2:'氣象站2',
                                3:'氣象站2距離',
                                4:'氣象站3',
                                5:'氣象站3距離',
                                })
    print('已合併與車禍點最近的三個天氣站點名稱')
    return data

In [10]:
negitive_df = weatherCombine(data=negitive_df,weather=weather)

開始讀取天氣站名與站點位置
已取出有天氣資料的站點，以及站點的位置資料
開始將有天氣資料的站點建立KDtree
開始計算每個車禍點最近的3個氣象站點
車禍最近的3個氣象站點:已計算100000筆| 車禍最近的3個氣象站點:已計算200000筆| 車禍最近的3個氣象站點:已計算300000筆| 車禍最近的3個氣象站點:已計算400000筆| 車禍最近的3個氣象站點:已計算500000筆| 車禍最近的3個氣象站點:已計算600000筆| 車禍最近的3個氣象站點:已計算700000筆| 車禍最近的3個氣象站點:已計算800000筆| 車禍最近的3個氣象站點:已計算900000筆| 車禍最近的3個氣象站點:已計算1000000筆| 車禍最近的3個氣象站點:已計算1100000筆| 車禍最近的3個氣象站點:已計算1200000筆| 車禍最近的3個氣象站點:已計算1300000筆| 車禍最近的3個氣象站點:已計算1400000筆| 車禍最近的3個氣象站點:已計算1500000筆| 車禍最近的3個氣象站點:已計算1600000筆| 車禍最近的3個氣象站點:已計算1700000筆| 車禍最近的3個氣象站點:已計算1800000筆| 車禍最近的3個氣象站點:已計算1900000筆| 車禍最近的3個氣象站點:已計算2000000筆| 車禍最近的3個氣象站點:已計算2100000筆| 車禍最近的3個氣象站點:已計算2200000筆| 車禍最近的3個氣象站點:已計算2300000筆| 車禍最近的3個氣象站點:已計算2400000筆| 車禍最近的3個氣象站點:已計算2500000筆| 車禍最近的3個氣象站點:已計算2600000筆| 車禍最近的3個氣象站點:已計算2700000筆| 車禍最近的3個氣象站點:已計算2800000筆| 車禍最近的3個氣象站點:已計算2900000筆| 車禍最近的3個氣象站點:已計算3000000筆| 車禍最近的3個氣象站點:已計算3100000筆| 車禍最近的3個氣象站點:已計算3200000筆| 車禍最近的3個氣象站點:已計算3300000筆| 車禍最近的3個氣象站點:已計算3400000筆| 車禍最近的3個氣象站點:已計算3500000筆| 車禍最近的3個氣象站點:已計算3600000筆| 車禍最近的3個氣象站點:已計算3700000筆| 車禍最近的3個氣象

In [11]:
negitive_df.to_csv('neg_data(no traffic attribute).csv',encoding='utf-8-sig',mode='w',index=None)

In [12]:
negitive_df = pd.read_csv('neg_data(no traffic attribute).csv', encoding='utf-8-sig', low_memory=False)

# 合併三個最近的氣站天氣資料，到事故資料中

In [13]:
def concatWeather(data:pd.DataFrame, weather:pd.DataFrame) -> pd.DataFrame:
    # 合併三個最近天氣站的天氣資料
    print('開始合併各天氣站的天氣資料')
    data['發生年度'] = data['發生年度'].astype(str) 
    data['發生月份'] = data['發生月份'].astype(str).apply(lambda x: x if len(x)>=2 else '0'+x)
    data['發生日期'] = data['發生日期'].astype(str).apply(lambda x: x if len(x)>=2 else '0'+x)
    data['發生時間'] = data['發生時間'].astype(str).apply(lambda x: x if len(x)>=2 else '0'+x)
    
    print('開始合併最近的天氣站天氣資料')
    data['weather_concat_station1'] =  data['發生年度']+data['發生月份']+data['發生日期']+data['發生時間']+data['氣象站1']
    data = data.merge(weather.rename(columns=dict(zip(weather.columns, weather.columns+'_station1'))), how='left', left_on='weather_concat_station1', right_on='weather_concat_station1')
    
    print('開始合併第二近的天氣站天氣資料')
    data['weather_concat_station2'] = data['發生年度']+data['發生月份']+data['發生日期']+data['發生時間']+data['氣象站2']
    data = data.merge(weather.rename(columns=dict(zip(weather.columns, weather.columns+'_station2'))), how='left', left_on='weather_concat_station2', right_on='weather_concat_station2')
    
    print('開始合併第三近的天氣站天氣資料')
    data['weather_concat_station3'] = data['發生年度']+data['發生月份']+data['發生日期']+data['發生時間']+data['氣象站3']
    data = data.merge(weather.rename(columns=dict(zip(weather.columns, weather.columns+'_station3'))), how='left', left_on='weather_concat_station3', right_on='weather_concat_station3')
    
    print('天氣資料合併完成')
    
    N = data[(data['氣溫(℃)_station1'].isnull()==True)].shape[0]
    print(f'共有 {N}筆車禍資料的三個天氣站均無氣溫資料')
    
    return data

In [14]:
negitive_df = concatWeather(data=negitive_df, weather=weather) 

開始合併各天氣站的天氣資料
開始合併最近的天氣站天氣資料
開始合併第二近的天氣站天氣資料
開始合併第三近的天氣站天氣資料
天氣資料合併完成
共有 448181筆車禍資料的三個天氣站均無氣溫資料


In [15]:
negitive_df.to_csv('neg_data(no traffic attribute).csv',encoding='utf-8-sig',mode='w',index=None)

In [16]:
negitive_df = pd.read_csv('neg_data(no traffic attribute).csv', encoding='utf-8-sig', low_memory=False)

# 篩選最近的三個氣站: 選取最近且有氣溫資料的氣站資料

In [17]:
def weather_station_chose(x:pd.Series) -> pd.Series:
    row = x[14:36] if type(x[22])!=float  else x[37:59] if type(x[45])!=float else x[60:82]
    row = row.rename(dict(zip(list(row.index), list(x.index[14:36]))))
    return row

In [18]:
weather_station1_filled = negitive_df.apply(weather_station_chose, axis=1)
weather_station1_filled = weather_station1_filled.replace('...', np.nan).replace('X', np.nan).replace('/', np.nan).replace('V', np.nan).replace('T', np.nan).replace('&', np.nan)

In [19]:
negitive_df = pd.concat([negitive_df.iloc[:,:14], weather_station1_filled], axis=1)
# weather_station1_filled 

In [20]:
negitive_df.to_csv('neg_data(no traffic attribute).csv',encoding='utf-8-sig',mode='w',index=None)

In [21]:
negitive_df = pd.read_csv('neg_data(no traffic attribute).csv', encoding='utf-8-sig', low_memory=False)

# 將天氣資料表，依照年月日時縣市做聚合

In [22]:
#天氣資料表上沒有地址(所以沒有縣市位置資料)，先將站點地址合併上去。合併後，地址的欄位名稱為location_y
station = pd.read_csv('./合併天氣/station(hasWeatherData).csv')
weather = weather.merge(station, left_on='location',right_on='name',how='left')

In [23]:
#天氣資料表中，date欄位的資料格式為20180101。觀測時間的資料格式為01
#把location_y中的縣市篩選出來，放到city欄位中，並把它和data、觀測時間相加，變成欄位time_city:格式20180101台北市
def _(l):
    if str(type(l))=="<class 'list'>":
        return l[0]
    else:
        return

weather['city'] = weather['location_y'].str.findall('..[市]|..[縣]').apply(_)    
weather['time_city'] = weather['date'].astype(str)+weather['觀測時間(hour)'].astype(str)+weather['city']   

In [24]:
#weather的5-21欄位為天氣相關特徵，-1為time_city。取出只包含這些欄位表，命名為weatherPerDay
# 把weatherPerDay除了time_city外的欄位下，若資料有奇怪文字，改為nan
use = [i if i != 22 else -1 for i in range(5,23)]
weatherPerDay = weather.iloc[:,use]
weatherPerDay.iloc[:,:-1] = weatherPerDay.iloc[:,:-1].replace('...', np.nan).replace('X', np.nan).replace('/', np.nan).replace('V', np.nan).replace('T', np.nan).replace('&', np.nan).astype('float')

C:\Users\student\AppData\Local\Temp\ipykernel_7016\1641148096.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weatherPerDay.iloc[:,:-1] = weatherPerDay.iloc[:,:-1].replace('...', np.nan).replace('X', np.nan).replace('/', np.nan).replace('V', np.nan).replace('T', np.nan).replace('&', np.nan).astype('float')


In [25]:
#把weatherPerDay依照time_city做聚合
weatherPerDay = weatherPerDay.groupby('time_city').mean()

In [26]:
weatherPerDay.to_csv('./合併天氣/weatherPerDay.csv',encoding='utf-8-sig',mode='w')

In [27]:
weatherPerDay = pd.read_csv('./合併天氣/weatherPerDay.csv', encoding='utf-8-sig', low_memory=False)

# 篩選事故資料和縣市天氣中，與天氣有關的欄位。讓兩個資料表的天氣相關欄位相同

In [28]:
# 負樣本資料表中，14欄後全為天氣站1的相關特徵
# 天氣站1的相關特徵名稱，後面仍有'_station1'字串的標註，這邊把它去掉
names_unchanged = list(negitive_df.columns[14:])
names_changed = [i[:-9] for i in names_unchanged]
negitive_df = negitive_df.rename(columns=dict(zip(names_unchanged, names_changed)))

In [29]:
# 要取出的天氣欄位如下: ['氣溫(℃)', '測站氣壓(hPa)','相對溼度(%)', '觀測時間(hour)', '降水量(mm)', '風向(360degree)','風速(m/s)']
# 先篩選負樣本資料表
weather_x_list = ['氣溫(℃)', '測站氣壓(hPa)','相對溼度(%)', '降水量(mm)', '風向(360degree)','風速(m/s)']
negitive_df = pd.concat([negitive_df.iloc[:,:14],negitive_df[weather_x_list]], axis=1)

In [30]:
# 篩選聚合後的縣市天氣資料表(留下time_city，和要的天氣特徵)
weatherPerDay = weatherPerDay[['time_city']+weather_x_list]

In [31]:
weatherPerDay.to_csv('./合併天氣/weatherPerDay.csv',encoding='utf-8-sig',mode='w')

In [32]:
weatherPerDay = pd.read_csv('./合併天氣/weatherPerDay.csv', encoding='utf-8-sig', low_memory=False)

# 將聚合後的縣市天氣資料，和事故點合併

In [33]:
#部分負樣本事故資料的地址為nan，剔除掉這些row
negitive_df = negitive_df[negitive_df.location.isnull()==False]
#在負樣本事故資料中，也做出一個time_city欄位:格式20180101台北市
negitive_df['time_city'] = negitive_df.發生年度.astype(str)+negitive_df.發生月份.astype(str)+negitive_df.發生日期.astype(str)+ negitive_df.發生時間.astype(str)+negitive_df.location.str.findall('..[市縣]').apply(lambda l: l[0])

In [34]:
#合併weatherPerDay和事故資料 #合併完後，相同名稱的天氣特徵欄名稱，會變為原名稱+_x(氣象站1)和原名稱+_y(縣市當天天氣)
negitive_df = negitive_df.merge(weatherPerDay, how='left', on='time_city')

In [35]:
#把氣象站1的資料中，若有遺漏值，用當天縣市天氣資料補上
#目前負樣本資料表中，欄位14~19為氣象站1資料，欄位21~26為氣象站2的資料
def weatherfill(x: pd.Series) -> pd.Series:
    data = x[14:20]
    if data.isnull().values.any():
        nanMask = np.where(data.isnull())[0]   # np.where(data.isnull())會回傳一個tuple，index=0為nan的位置的np.array，index=1為dtype
        data_city = x[21:27]
        data[nanMask] = data_city[nanMask]
    return data

# 把negitive_df的欄位14前的資料，和跑出來的新天氣資料表合併
negitive_df = pd.concat([negitive_df.iloc[:,:14], negitive_df.apply(weatherfill, axis=1)], axis=1)

In [36]:
negitive_df

,發生年度,發生月份,發生日期,發生時間,經度,緯度,name,location,氣象站1,氣象站1距離,氣象站2,氣象站2距離,氣象站3,氣象站3距離,氣溫(℃)_x,測站氣壓(hPa)_x,相對溼度(%)_x,降水量(mm)_x,風向(360degree)_x,風速(m/s)_x
0,2020,12,18,15,120.536702,23.703466,北港 (Beigang),雲林縣斗六市鎮北路345-1號(斗六消防分隊),北港 (Beigang),0.017593,林內 (Linnei),0.063436,元長 (Yuanchang),0.067121,20.8,1014.0,79.0,0.0,357.0,4.1
1,2021,5,5,21,121.513505,25.077870,天母 (Tianmu),臺北市士林區天母東路116號(三玉國小內),天母 (Tianmu),0.018672,澳底 (AODI),0.024283,石牌 (Shipai) (撤銷站),0.029296,21.0,1014.2,80.0,0.0,146.0,0.9
2,2021,10,22,22,121.663959,24.949437,火燒寮 (Huoshaoliao),新北市平溪區東勢里,火燒寮 (Huoshaoliao),0.044476,四堵 (Sihdu),0.047125,福山植物園 (Fushan Botanical Garden),0.064098,17.0,990.4,100.0,2.0,31.0,1.7
3,2021,12,21,11,121.279105,24.633933,信義 (Xinyi),臺北市信義區市府路1號(台北市政府頂樓),關西 (Guanxi),0.083388,多加屯 (Duojiatun),0.140096,雪霸 (Xueba),0.179625,17.9,997.9,97.0,0.5,0.0,0.0
4,2019,4,7,12,120.489808,22.680644,赤山 (Chishan),屏東縣屏東市復興路245號(台灣糖業公司屏東區處場區內),赤山 (Chishan),0.020869,南州 (Nanzhou),0.047826,九如 (Jiuru),0.054241,27.9,1011.1,67.0,0.0,244.0,3.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3989792,2019,10,14,14,120.620479,22.551515,春日 (Chunri),屏東縣萬巒鄉新隆路64號萬巒農場（佳佐段180號）,春日 (Chunri),0.024652,潮州 (Chaojhou),0.041251,墾雷 (Kenlei),0.057547,26.5,1006.5,84.0,0.0,205.0,0.4
3989793,2022,6,22,17,120.711020,23.925731,梅峰 (Meifeng),南投縣南投市龍泉里玉井街5號(南投市公所樓頂),梅峰 (Meifeng),0.027713,昆陽 (Kunyang),0.056610,草屯 (Caotun),0.069103,16.3,791.2,99.0,2.0,166.0,0.9
3989794,2020,9,23,10,120.576133,23.759012,莿桐 (Citong),雲林縣林內鄉中正路685號(林內消防分隊),莿桐 (Citong),0.035049,北港 (Beigang),0.052062,花壇 (Huatan) (撤銷站),0.058599,30.0,1004.4,66.0,0.0,282.0,0.9
3989795,2022,9,3,0,120.691294,24.032694,昆陽 (Kunyang),南投縣草屯鎮中山里草鞋墩1街8號(草屯鎮公所樓頂),昆陽 (Kunyang),0.060018,潭子 (Tanzi),0.060539,鹿港 (Lukang),0.072051,11.6,697.4,100.0,2.5,197.0,2.4


In [37]:
negitive_df1 = negitive_df

In [ ]:
negitive_df.to_csv('neg_data(no traffic attribute).csv',encoding='utf-8-sig',mode='w',index=None)

In [ ]:
negitive_df = pd.read_csv('neg_data(no traffic attribute).csv', encoding='utf-8-sig', low_memory=False)

# 合併人口密度

In [38]:
def density(df: pd.DataFrame)->pd.DataFrame:
    print('開始合併人口密度資料')
    df['市/縣區'] = df["location"].str.findall('..[市].*?[區]|..[縣].*?[鄉鎮市]')
    def _(l):
        try:
            l[0]
            return l[0]
        except Exception:
            return None

    df['市/縣區'] = df['市/縣區'].apply(_)

    df['年/地區'] = df['發生年度'].astype(str)+df['市/縣區']

    files = glob('./合併人口密度/opendata*N010.csv')

    df1 = pd.concat(
        (pd.read_csv(file, usecols=['statistic_yyy','site_id','population_density'], dtype={ 'statistic_yyy': str, 'site_id':str, 'population_density':str}) for file in files), ignore_index=True)
    df1.dropna(inplace=True)
    df1 = df1[df1['population_density']!='…']
    df1 = df1[df1['statistic_yyy']!='統計年']
    df1['西元年'] = df1['statistic_yyy'].map(int)+1912
    df1['年/地區'] = df1['西元年'].map(str)+df1['site_id']
    df = pd.merge(df, 
                 df1, 
                 on ='年/地區', 
                 how ='left')
    print('人口密度資料合併完成')
    return df

In [39]:
negitive_df = density(negitive_df)

開始合併人口密度資料
人口密度資料合併完成


In [ ]:
negitive_df.to_csv('neg_data(no traffic attribute).csv',encoding='utf-8-sig',mode='w',index=None)

In [ ]:
negitive_df = pd.read_csv('neg_data(no traffic attribute).csv', encoding='utf-8-sig', low_memory=False)

# 合併車流量

In [40]:
def streamCombine(data:pd.DataFrame)->pd.DataFrame:
    print('開始合併車流資料')
    stream = pd.read_csv(f'合併車流量/106-110交通流量彙整.csv', low_memory=False)
    stream['總計_流量(PCU)'] = stream['總計_流量(PCU)'].str.findall('\d').str.join('').astype(int)
    stream['ID']=stream['年份'].astype(str)+stream['調查站_座標_經度'].astype(str)+stream['調查站_座標_緯度'].astype(str)
    stream = stream.groupby('ID').mean()
    
    print('建立車流站點KDTree')
    KDTree06 = spatial.KDTree(stream[stream['年份']==106][['調查站_座標_經度','調查站_座標_緯度']])
    KDTree07 = spatial.KDTree(stream[stream['年份']==107][['調查站_座標_經度','調查站_座標_緯度']])
    KDTree08 = spatial.KDTree(stream[stream['年份']==108][['調查站_座標_經度','調查站_座標_緯度']])
    KDTree09 = spatial.KDTree(stream[stream['年份']==109][['調查站_座標_經度','調查站_座標_緯度']])
    KDTree10 = spatial.KDTree(stream[stream['年份']==110][['調查站_座標_經度','調查站_座標_緯度']])
    global c
    c=0
    def nearest(x: pd.DataFrame):
        if x.發生年度 == 2022:                      #事故民國111年，流量民國110年
            distance = KDTree10.query((x.經度, x.緯度),k=3)[0]
            index = KDTree10.query((x.經度, x.緯度),k=3)[1]
        elif x.發生年度 == 2021:                    #事故民國110年，流量民國109年
            distance = KDTree09.query((x.經度, x.緯度),k=3)[0]
            index = KDTree09.query((x.經度, x.緯度),k=3)[1]
        elif x.發生年度 == 2020:                    #事故民國109年，流量民國108年
            distance = KDTree08.query((x.經度, x.緯度),k=3)[0]
            index = KDTree08.query((x.經度, x.緯度),k=3)[1]
        elif x.發生年度 == 2019:                    #事故民國108年，流量民國107年
            distance = KDTree07.query((x.經度, x.緯度),k=3)[0]
            index = KDTree07.query((x.經度, x.緯度),k=3)[1]
        else:                                      #事故民國107年，流量民國106年
            distance = KDTree06.query((x.經度, x.緯度),k=3)[0]
            index = KDTree06.query((x.經度, x.緯度),k=3)[1]
        df = stream.iloc[index,:]
        df['與事故點距離'] = pd.Series(distance,index=df.index)
        df = df[['調查站_座標_經度','調查站_座標_緯度','與事故點距離','總計_流量(PCU)']]
        df = pd.concat([df.iloc[0,],df.iloc[1,],df.iloc[2,]])

        global c
        c+=1
        if c%100000==0:
            print(f"車禍點最近的車流站點:已計算{c}筆",'|',' ', end='')
        return df
    data = pd.concat([data, data.apply(nearest, axis=1)], axis = 1)
    data.columns = list(data.columns[:-12])+['調查站1_座標_經度','調查站1_座標_緯度','調查站1與事故點距離','調查站1總計_流量(PCU)','調查站2_座標_經度','調查站2_座標_緯度','調查站2與事故點距離','調查站2總計_流量(PCU)','調查站3_座標_經度','調查站3_座標_緯度','調查站3與事故點距離','調查站3總計_流量(PCU)']
    
    print('車流資料合併完成')
    return data

In [41]:
negitive_df = streamCombine(negitive_df)

開始合併車流資料
建立車流站點KDTree


C:\Users\student\AppData\Local\Temp\ipykernel_7016\1225651620.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  stream = stream.groupby('ID').mean()


車禍點最近的車流站點:已計算100000筆 |  車禍點最近的車流站點:已計算200000筆 |  車禍點最近的車流站點:已計算300000筆 |  車禍點最近的車流站點:已計算400000筆 |  車禍點最近的車流站點:已計算500000筆 |  車禍點最近的車流站點:已計算600000筆 |  車禍點最近的車流站點:已計算700000筆 |  車禍點最近的車流站點:已計算800000筆 |  車禍點最近的車流站點:已計算900000筆 |  車禍點最近的車流站點:已計算1000000筆 |  車禍點最近的車流站點:已計算1100000筆 |  車禍點最近的車流站點:已計算1200000筆 |  車禍點最近的車流站點:已計算1300000筆 |  車禍點最近的車流站點:已計算1400000筆 |  車禍點最近的車流站點:已計算1500000筆 |  車禍點最近的車流站點:已計算1600000筆 |  車禍點最近的車流站點:已計算1700000筆 |  車禍點最近的車流站點:已計算1800000筆 |  車禍點最近的車流站點:已計算1900000筆 |  車禍點最近的車流站點:已計算2000000筆 |  車禍點最近的車流站點:已計算2100000筆 |  車禍點最近的車流站點:已計算2200000筆 |  車禍點最近的車流站點:已計算2300000筆 |  車禍點最近的車流站點:已計算2400000筆 |  車禍點最近的車流站點:已計算2500000筆 |  車禍點最近的車流站點:已計算2600000筆 |  車禍點最近的車流站點:已計算2700000筆 |  車禍點最近的車流站點:已計算2800000筆 |  車禍點最近的車流站點:已計算2900000筆 |  車禍點最近的車流站點:已計算3000000筆 |  車禍點最近的車流站點:已計算3100000筆 |  車禍點最近的車流站點:已計算3200000筆 |  車禍點最近的車流站點:已計算3300000筆 |  車禍點最近的車流站點:已計算3400000筆 |  車禍點最近的車流站點:已計算3500000筆 |  車禍點最近的車流站點:已計算3600000筆 |  車禍點最近的車流站點:已計算3700000筆 |  車禍點最近的車流站點:已計算3800000筆 |  車禍點最近的車流站點:已計算3900000

In [42]:
negitive_df2 = negitive_df

In [ ]:
negitive_df.to_csv('neg_data(no traffic attribute).csv',encoding='utf-8-sig',mode='w',index=None)

In [ ]:
negitive_df = pd.read_csv('neg_data(no traffic attribute).csv', encoding='utf-8-sig', low_memory=False)

# 計算車流量加權

In [43]:
def streamWeited(df):
    # 有三個站點的車流，並有計算三個站點的距離。參考克力金法，用反距離加權計算事故點車流
    # 反距離加權: ((1/某站點距離)*某站點車流) / 加總所有站點的(1/某站點距離)  
    df['流量加權'] = df['調查站1總計_流量(PCU)']*(1/(df.調查站1與事故點距離))/(1/(df.調查站1與事故點距離)+1/(df.調查站2與事故點距離)+1/(df.調查站3與事故點距離)) +\
                df['調查站2總計_流量(PCU)']*(1/(df.調查站2與事故點距離))/(1/(df.調查站1與事故點距離)+1/(df.調查站2與事故點距離)+1/(df.調查站3與事故點距離)) +\
                df['調查站3總計_流量(PCU)']*(1/(df.調查站3與事故點距離))/(1/(df.調查站1與事故點距離)+1/(df.調查站2與事故點距離)+1/(df.調查站3與事故點距離))
    return df

In [44]:
negitive_df = streamWeited(negitive_df)

In [45]:
negitive_df

,發生年度,發生月份,發生日期,發生時間,經度,緯度,name,location,氣象站1,氣象站1距離,...,調查站1總計_流量(PCU),調查站2_座標_經度,調查站2_座標_緯度,調查站2與事故點距離,調查站2總計_流量(PCU),調查站3_座標_經度,調查站3_座標_緯度,調查站3與事故點距離,調查站3總計_流量(PCU),流量加權
0,2020,12,18,15,120.536702,23.703466,北港 (Beigang),雲林縣斗六市鎮北路345-1號(斗六消防分隊),北港 (Beigang),0.017593,...,22397.5,120.39494,24.04355,0.029716,11347.5,120.36143,22.48551,0.031824,4589.0,13124.064104
1,2021,5,5,21,121.513505,25.077870,天母 (Tianmu),臺北市士林區天母東路116號(三玉國小內),天母 (Tianmu),0.018672,...,689.5,121.44910,25.06700,0.028832,10926.5,121.42542,25.06910,0.033900,29212.5,9938.311810
2,2021,10,22,22,121.663959,24.949437,火燒寮 (Huoshaoliao),新北市平溪區東勢里,火燒寮 (Huoshaoliao),0.044476,...,6854.5,121.74062,24.26842,0.049135,4997.5,121.76425,24.77353,0.073258,5093.5,5910.896902
3,2021,12,21,11,121.279105,24.633933,信義 (Xinyi),臺北市信義區市府路1號(台北市政府頂樓),關西 (Guanxi),0.083388,...,27596.0,121.14184,25.06416,0.170910,10471.5,121.12232,24.56572,0.178843,15883.5,19306.252201
4,2019,4,7,12,120.489808,22.680644,赤山 (Chishan),屏東縣屏東市復興路245號(台灣糖業公司屏東區處場區內),赤山 (Chishan),0.020869,...,16791.0,120.39416,23.57142,0.150849,8454.5,120.37190,24.21210,0.226740,22397.5,15124.156391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3989792,2019,10,14,14,120.620479,22.551515,春日 (Chunri),屏東縣萬巒鄉新隆路64號萬巒農場（佳佐段180號）,春日 (Chunri),0.024652,...,302.0,120.39416,23.57142,0.238041,8454.5,120.39492,24.04356,0.240582,16791.0,8336.942344
3989793,2022,6,22,17,120.711020,23.925731,梅峰 (Meifeng),南投縣南投市龍泉里玉井街5號(南投市公所樓頂),梅峰 (Meifeng),0.027713,...,31798.0,120.48157,24.29197,0.021567,6507.5,120.46247,24.37300,0.034740,1194.0,16114.008396
3989794,2020,9,23,10,120.576133,23.759012,莿桐 (Citong),雲林縣林內鄉中正路685號(林內消防分隊),莿桐 (Citong),0.035049,...,11347.5,120.42493,22.04327,0.058851,10351.0,120.36143,22.48551,0.062955,4589.0,9085.433554
3989795,2022,9,3,0,120.691294,24.032694,昆陽 (Kunyang),南投縣草屯鎮中山里草鞋墩1街8號(草屯鎮公所樓頂),昆陽 (Kunyang),0.060018,...,5525.5,120.46247,24.37304,0.027064,3410.0,120.46455,24.26439,0.035791,4725.0,4610.471958


In [ ]:
negitive_df.to_csv('neg_data(no traffic attribute).csv',encoding='utf-8-sig',mode='w',index=None)

In [ ]:
negitive_df = pd.read_csv('neg_data(no traffic attribute).csv', encoding='utf-8-sig', low_memory=False)

# 格式弄成同正樣本

In [46]:
# 正樣本column: ['發生年度', '發生月份', '發生日期', '發生時間', '經度', '緯度', '氣溫(℃)_x', '測站氣壓(hPa)_x','相對溼度(%)_x', '觀測時間(hour)_x', '降水量(mm)_x', '風向(360degree)_x','風速(m/s)_x', 'population_density', '流量加權']

In [47]:
negitive_df = negitive_df[ ['發生年度', '發生月份', '發生日期', '發生時間', '經度', '緯度', '氣溫(℃)_x', '測站氣壓(hPa)_x','相對溼度(%)_x',  '降水量(mm)_x', '風向(360degree)_x','風速(m/s)_x', 'population_density', '流量加權']]

In [48]:
negitive_df = negitive_df[negitive_df.isnull().sum(axis=1)==0]

In [49]:
negitive_df

,發生年度,發生月份,發生日期,發生時間,經度,緯度,氣溫(℃)_x,測站氣壓(hPa)_x,相對溼度(%)_x,降水量(mm)_x,風向(360degree)_x,風速(m/s)_x,population_density,流量加權
0,2020,12,18,15,120.536702,23.703466,20.8,1014.0,79.0,0.0,357.0,4.1,1159,13124.064104
1,2021,5,5,21,121.513505,25.077870,21.0,1014.2,80.0,0.0,146.0,0.9,4465,9938.311810
2,2021,10,22,22,121.663959,24.949437,17.0,990.4,100.0,2.0,31.0,1.7,62,5910.896902
3,2021,12,21,11,121.279105,24.633933,17.9,997.9,97.0,0.5,0.0,0.0,19205,19306.252201
4,2019,4,7,12,120.489808,22.680644,27.9,1011.1,67.0,0.0,244.0,3.9,3068,15124.156391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3989792,2019,10,14,14,120.620479,22.551515,26.5,1006.5,84.0,0.0,205.0,0.4,339,8336.942344
3989793,2022,6,22,17,120.711020,23.925731,16.3,791.2,99.0,2.0,166.0,0.9,1372,16114.008396
3989794,2020,9,23,10,120.576133,23.759012,30.0,1004.4,66.0,0.0,282.0,0.9,473,9085.433554
3989795,2022,9,3,0,120.691294,24.032694,11.6,697.4,100.0,2.5,197.0,2.4,932,4610.471958


In [51]:
negitive_df.to_csv('neg_data(no traffic attribute).csv',encoding='utf-8-sig',mode='w',index=None)

In [52]:
negitive_df = pd.read_csv('neg_data(no traffic attribute).csv', encoding='utf-8-sig', low_memory=False)

In [53]:
negitive_df['車禍發生']=0

In [56]:
negitive_df

,發生年度,發生月份,發生日期,發生時間,經度,緯度,氣溫(℃)_x,測站氣壓(hPa)_x,相對溼度(%)_x,降水量(mm)_x,風向(360degree)_x,風速(m/s)_x,population_density,流量加權,車禍發生
0,2020,12,18,15,120.536702,23.703466,20.8,1014.0,79.0,0.0,357.0,4.1,1159,13124.064104,0
1,2021,5,5,21,121.513505,25.077870,21.0,1014.2,80.0,0.0,146.0,0.9,4465,9938.311810,0
2,2021,10,22,22,121.663959,24.949437,17.0,990.4,100.0,2.0,31.0,1.7,62,5910.896902,0
3,2021,12,21,11,121.279105,24.633933,17.9,997.9,97.0,0.5,0.0,0.0,19205,19306.252201,0
4,2019,4,7,12,120.489808,22.680644,27.9,1011.1,67.0,0.0,244.0,3.9,3068,15124.156391,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3704727,2019,10,14,14,120.620479,22.551515,26.5,1006.5,84.0,0.0,205.0,0.4,339,8336.942344,0
3704728,2022,6,22,17,120.711020,23.925731,16.3,791.2,99.0,2.0,166.0,0.9,1372,16114.008396,0
3704729,2020,9,23,10,120.576133,23.759012,30.0,1004.4,66.0,0.0,282.0,0.9,473,9085.433554,0
3704730,2022,9,3,0,120.691294,24.032694,11.6,697.4,100.0,2.5,197.0,2.4,932,4610.471958,0


In [59]:
negitive_df.to_csv('../2.機器學習/無交通相關特徵資料集/neg_data(no traffic attribute).csv',encoding='utf-8-sig',mode='w',index=None)

In [58]:
pattern='X'
negitive_df[negitive_df.apply((lambda x: x.astype(str).str.contains(pattern).any()),axis=1)]

,發生年度,發生月份,發生日期,發生時間,經度,緯度,氣溫(℃)_x,測站氣壓(hPa)_x,相對溼度(%)_x,降水量(mm)_x,風向(360degree)_x,風速(m/s)_x,population_density,流量加權,車禍發生


# 加入day of week、day of year

In [4]:
negitive_df = pd.read_csv('neg_data(no traffic attribute).csv', encoding='utf-8-sig', low_memory=False)

In [5]:
negitive_df 

,發生年度,發生月份,發生日期,發生時間,經度,緯度,氣溫(℃)_x,測站氣壓(hPa)_x,相對溼度(%)_x,降水量(mm)_x,風向(360degree)_x,風速(m/s)_x,population_density,流量加權
0,2020,12,18,15,120.536702,23.703466,20.8,1014.0,79.0,0.0,357.0,4.1,1159,13124.064104
1,2021,5,5,21,121.513505,25.077870,21.0,1014.2,80.0,0.0,146.0,0.9,4465,9938.311810
2,2021,10,22,22,121.663959,24.949437,17.0,990.4,100.0,2.0,31.0,1.7,62,5910.896902
3,2021,12,21,11,121.279105,24.633933,17.9,997.9,97.0,0.5,0.0,0.0,19205,19306.252201
4,2019,4,7,12,120.489808,22.680644,27.9,1011.1,67.0,0.0,244.0,3.9,3068,15124.156391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3704727,2019,10,14,14,120.620479,22.551515,26.5,1006.5,84.0,0.0,205.0,0.4,339,8336.942344
3704728,2022,6,22,17,120.711020,23.925731,16.3,791.2,99.0,2.0,166.0,0.9,1372,16114.008396
3704729,2020,9,23,10,120.576133,23.759012,30.0,1004.4,66.0,0.0,282.0,0.9,473,9085.433554
3704730,2022,9,3,0,120.691294,24.032694,11.6,697.4,100.0,2.5,197.0,2.4,932,4610.471958


In [8]:
date = negitive_df.apply((lambda x: x.發生年度.astype(int).astype(str)+'-'+ x.發生月份.astype(int).astype(str)+'-'+x.發生日期.astype(int).astype(str)), axis = 1)

In [18]:
# 週一為0，週日為6
dow = pd.to_datetime(date).dt.dayofweek
negitive_df['dow'] = dow

In [20]:
doy = pd.to_datetime(date).dt.dayofyear
negitive_df['doy'] = doy

# 加入107-111假期單熱編碼

In [83]:
def taiwan_festival(x: int) -> pd.Series:
    
    sr = pd.Series([0,0,0,0,0,0,0],index=['元旦','春節','二二八','兒童清明','端午','中秋','雙十'])
    
    def _2018( x, sr):
        if x == 20180101:
            sr['元旦'] = 1
        elif 20180215 <= x <= 20180220:
            sr['春節'] = 1
        elif x == 20180228:
            sr['二二八'] = 1
        elif 20180404 <= x <= 20180408:
            sr['兒童清明'] = 1
        elif 20180616 <= x <= 20180618:
            sr['端午'] = 1
        elif 20180922 <= x <= 20180924:
            sr['中秋'] = 1
        elif x == 20181010:
            sr['雙十'] = 1
        elif 20181229 <= x <= 20181231:
            sr['元旦'] = 1
        return sr

    def _2019( x, sr):
        if x == 20190101:
            sr['元旦'] = 1
        elif 20190202 <= x <= 20190210:
            sr['春節'] = 1
        elif 20190228 <= x <= 20190303:
            sr['二二八'] = 1
        elif 20190404 <= x <= 20190407:
            sr['兒童清明'] = 1
        elif 20190607 <= x <= 20190609:
            sr['端午'] = 1
        elif 20190913 <= x <= 20190915:
            sr['中秋'] = 1
        elif 20191010 <= x <= 20191013:
            sr['雙十'] = 1
        return sr


    def _2020( x, sr):
        if x == 20200101:
            sr['元旦'] = 1
        elif 20200123 <= x <= 20200129:
            sr['春節'] = 1
        elif 20200228 <= x <= 20200301:
            sr['二二八'] = 1
        elif 20200402 <= x <= 20200405:
            sr['兒童清明'] = 1
        elif 20200625 <= x <= 20200628:
            sr['端午'] = 1
        elif 20201001 <= x <= 20201004:
            sr['中秋'] = 1
        elif 20201009 <= x <= 20201011:
            sr['雙十'] = 1
        return sr

    def _2021( x, sr):
        if 20210101 <= x <= 20210103:
            sr['元旦'] = 1
        elif 20210210  <= x <= 20210216:
            sr['春節'] = 1
        elif 20210227 <= x <= 20210301:
            sr['二二八'] = 1
        elif 20210402 <= x <= 20210405:
            sr['兒童清明'] = 1
        elif 20210612 <= x <= 20210614:
            sr['端午'] = 1
        elif 20210918 <= x <= 20210921:
            sr['中秋'] = 1
        elif 20211009 <= x <= 20211011:
            sr['雙十'] = 1
        elif x == 20211231:
            sr['元旦'] = 1
        return sr

    def _2022( x, sr):
        if 20220101 <= x <= 20220102:
            sr['元旦'] = 1
        elif 20220129 <= x <= 20220206:
            sr['春節'] = 1
        elif 20220226 <= x <= 20220228:
            sr['二二八'] = 1
        elif 20220402 <= x <= 20220405:
            sr['兒童清明'] = 1
        elif 20220603 <= x <= 20220605:
            sr['端午'] = 1
        elif 20220909 <= x <= 20220911:
            sr['中秋'] = 1
        elif 20221008 <= x <= 20221010:
            sr['雙十'] = 1
        elif x == 20221231:
            sr['元旦'] = 1
        return sr

    year = x // 10000
    
    if year == 2018:
        sr = _2018(x, sr)
    elif year == 2019:
        sr = _2019(x, sr)
    elif year == 2020:
        sr = _2020(x, sr)
    elif year == 2021:
        sr = _2021(x, sr)
    elif year == 2022:
        sr = _2022(x, sr)
    
    return sr


In [84]:
date_int = negitive_df.發生年度.astype(int)*10000 + negitive_df.發生月份.astype(int)*100 + negitive_df.發生日期.astype(int)

In [87]:
from tqdm import tqdm
tqdm.pandas()
festival_df = date_int.progress_apply(taiwan_festival)

100%|██████████████████████████████████████████████████████████████████████| 3704732/3704732 [09:21<00:00, 6600.26it/s]


In [88]:
festival_df.sum()

元旦      25653
春節      78620
二二八     28999
兒童清明    43791
端午      33568
中秋      36047
雙十      30385
dtype: int64

In [91]:
negitive_df = pd.concat([festival_df, negitive_df], axis=1)

In [100]:
order_list = [ i for i in range(negitive_df.shape[1])]
order_list = [-2, -1] + order_list[:-2]

In [104]:
negitive_df = negitive_df.iloc[:,order_list]

In [ ]:
negitive_df.to_csv('neg_data(no traffic attribute + festival).csv',encoding='utf-8-sig',mode='w',index=None)

In [ ]:
negitive_df

In [5]:
negitive_df = pd.read_csv('neg_data(no traffic attribute + festival).csv', encoding='utf-8-sig', low_memory=False)

In [6]:
negitive_df['車禍發生'] = 0

In [8]:
negitive_df.to_csv('neg_data(no traffic attribute + festival).csv',encoding='utf-8-sig',mode='w',index=None)